# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [23]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


In [2]:
test = pd.read_csv('flights_test.csv')

In [3]:
df = pd.read_csv('flights_merged.csv', index_col=0)

In [4]:
df = df.sample(frac=0.01)

In [5]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,passengers,capacity,total_gallons,dep_delay_y,nas_delay_y,security_delay_y,late_aircraft_delay_y,weather_delay_y,carrier_delay_y
518617,2019-02-21,UA,UA,UA,448,UA,N39450,448,10721,BOS,...,1754.0,3508.571429,481.428571,2.975601e+08,37.281407,19.603015,0.000000,19.296482,2.502513,9.075377
492127,2019-04-11,DL,DL,DL,860,DL,N910DL,860,13931,ORF,...,516.0,5331.571429,824.428571,3.087060e+08,50.089947,13.502646,0.000000,21.322751,2.661376,14.439153
6661,2019-08-16,AA,AA,AA,2373,AA,N833AW,2373,11298,DFW,...,925.0,146.400000,29.600000,3.079852e+08,56.134078,5.614525,0.000000,24.865922,2.346369,23.122905
159024,2019-10-22,AA,AA_CODESHARE,AA,3241,OO,N770SK,3241,13930,ORD,...,621.0,3024.333333,315.333333,8.602255e+06,49.114894,11.395745,0.000000,19.821277,4.923404,17.621277
554206,2019-11-21,UA,UA,UA,386,UA,N66803,386,11292,DEN,...,1754.0,4611.000000,370.750000,2.975601e+08,56.534247,27.297945,0.000000,20.184932,4.821918,8.993151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210578,2019-08-01,UA,UA,UA,2223,UA,N64809,2223,11292,DEN,...,1024.0,2618.000000,240.000000,2.975601e+08,37.808050,16.321981,0.000000,18.684211,3.554180,8.092879
330074,2019-05-20,AA,AA,AA,792,AA,N352PS,792,11298,DFW,...,1303.0,7075.400000,865.700000,3.079852e+08,55.219697,10.293939,0.004545,19.810606,2.603030,24.628788
143938,2019-03-09,UA,UA,UA,2273,UA,N850UA,2273,13931,ORF,...,1553.0,1125.000000,203.000000,2.975601e+08,44.081633,9.653061,0.000000,0.000000,0.387755,38.551020
177276,2019-11-18,DL,DL_CODESHARE,DL,5725,CP,N621CZ,5725,12892,LAX,...,308.0,42.000000,102.000000,0.000000e+00,52.239437,5.881690,0.000000,25.225352,0.171831,22.608451


In [6]:
y= df['arr_delay']

In [7]:
X = df

In [8]:
X = X.drop(columns=['arr_delay'])

In [9]:
#X.fl_date = X.fl_date.apply(pd.to_datetime)

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [10]:
X.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time',
       'arr_time', 'cancelled', 'cancellation_code', 'diverted', 'dup',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'flights',
       'distance', 'passengers', 'capacity', 'total_gallons', 'dep_delay_y',
       'nas_delay_y', 'security_delay_y', 'late_aircraft_delay_y',
       'weather_delay_y', 'carrier_delay_y'],
      dtype='object')

In [10]:
X = X[['origin_airport_id', 'dest_airport_id', 'air_time', 'distance', 'passengers', 'crs_elapsed_time','capacity', 'total_gallons', 'dep_delay_y',
       'nas_delay_y', 'security_delay_y', 'late_aircraft_delay_y',
       'weather_delay_y', 'carrier_delay_y']]
#dropped flight date :( 

In [11]:
X

,origin_airport_id,dest_airport_id,air_time,flights,distance,passengers,crs_elapsed_time,capacity,total_gallons,dep_delay_y,nas_delay_y,security_delay_y,late_aircraft_delay_y,weather_delay_y,carrier_delay_y
518617,10721,11292,286.0,1.0,1754.0,3508.571429,284.0,481.428571,2.975601e+08,37.281407,19.603015,0.000000,19.296482,2.502513,9.075377
492127,13931,10397,86.0,1.0,516.0,5331.571429,113.0,824.428571,3.087060e+08,50.089947,13.502646,0.000000,21.322751,2.661376,14.439153
6661,11298,14685,118.0,1.0,925.0,146.400000,144.0,29.600000,3.079852e+08,56.134078,5.614525,0.000000,24.865922,2.346369,23.122905
159024,13930,10821,91.0,1.0,621.0,3024.333333,122.0,315.333333,8.602255e+06,49.114894,11.395745,0.000000,19.821277,4.923404,17.621277
554206,11292,10721,191.0,1.0,1754.0,4611.000000,228.0,370.750000,2.975601e+08,56.534247,27.297945,0.000000,20.184932,4.821918,8.993151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210578,11292,14747,138.0,1.0,1024.0,2618.000000,180.0,240.000000,2.975601e+08,37.808050,16.321981,0.000000,18.684211,3.554180,8.092879
330074,11298,14100,163.0,1.0,1303.0,7075.400000,195.0,865.700000,3.079852e+08,55.219697,10.293939,0.004545,19.810606,2.603030,24.628788
143938,13931,11292,226.0,1.0,1553.0,1125.000000,245.0,203.000000,2.975601e+08,44.081633,9.653061,0.000000,0.000000,0.387755,38.551020
177276,12892,14831,48.0,1.0,308.0,42.000000,80.0,102.000000,0.000000e+00,52.239437,5.881690,0.000000,25.225352,0.171831,22.608451


### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [12]:
#data_dmatrix = xgb.DMatrix(data=X,label=y, enable_categorical=True)

In [ ]:
#objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 7, alpha = 0.1,

In [24]:
scaler = StandardScaler()

In [25]:
X = pd.DataFrame(scaler.fit_transform(X))

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8)


In [27]:
lr = LinearRegression()

In [28]:
lr.fit(X_train, y_train) 

LinearRegression()

In [29]:
lr.score(X_train, y_train)

0.03548352555792911

In [30]:
lr.score(X_test, y_test)

0.030917921003375093

### Random Forest

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.